<a href="https://colab.research.google.com/github/mandliya/textvqa/blob/master/text_vqa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Install dependencies
!pip install ninja yacs cython matplotlib demjson
!pip install git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI

     |████████████████████████████████| 102kB 2.7MB/s 
     |████████████████████████████████| 133kB 8.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/c5/d2/ab/a54fb5ea53ac3badba098160e8452fa126a51febda80440ded
Successfully built demjson
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-kd5nxhnv
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-kd5nxhnv
  Stored in directory: /tmp/pip-ephem-wheel-cache-_qx8mg0c/wheels/90/51/41/646daf401c3bc408ff10de34ec76587a9b3ebfac8d21ca5c3a
Successfully built pycocotools


In [0]:
%cd /content/
%rm -rf fastText
!git clone https://github.com/facebookresearch/fastText.git fastText
%cd /content/fastText
!pip install -e .

/content
Cloning into 'fastText'...
remote: Enumerating objects: 2, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3197 (delta 0), reused 1 (delta 0), pack-reused 3195
Receiving objects: 100% (3197/3197), 7.84 MiB | 33.88 MiB/s, done.
Resolving deltas: 100% (2007/2007), done.
/content/fastText
Obtaining file:///content/fastText
  Running setup.py develop for fasttext


In [0]:
%cd /content/
%rm -rf pythia
!git clone https://github.com/facebookresearch/pythia.git pythia
%cd /content/pythia
# Don't modify torch version
!sed -i '/torch/d' requirements.txt
!pip install -e .
import sys
sys.path.append('/content/pythia')

/content
Cloning into 'pythia'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 3295 (delta 16), reused 42 (delta 14), pack-reused 3218
Receiving objects: 100% (3295/3295), 6.51 MiB | 20.26 MiB/s, done.
Resolving deltas: 100% (2129/2129), done.
/content/pythia
Obtaining file:///content/pythia
     |████████████████████████████████| 51kB 2.0MB/s 
     |████████████████████████████████| 61kB 22.8MB/s 
     |████████████████████████████████| 450kB 8.2MB/s 
     |████████████████████████████████| 194kB 44.4MB/s 
     |████████████████████████████████| 3.1MB 45.3MB/s 
     |████████████████████████████████| 71kB 22.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/97/8a/10/d646015f33c525688e91986c4544c68019b19a473cb33d3b55
Successfully built nltk
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: tq

In [0]:
# Install maskrcnn-benchmark to extract detectron features
%cd /content
!git clone https://gitlab.com/meetshah1995/vqa-maskrcnn-benchmark.git
%cd /content/vqa-maskrcnn-benchmark
# Compile custom layers and build mask-rcnn backbone
!python setup.py build
!python setup.py develop
sys.path.append('/content/vqa-maskrcnn-benchmark')

/content
Cloning into 'vqa-maskrcnn-benchmark'...
remote: Enumerating objects: 730, done.
remote: Counting objects: 100% (730/730), done.
remote: Compressing objects: 100% (337/337), done.
remote: Total 730 (delta 402), reused 701 (delta 386)
Receiving objects: 100% (730/730), 3.76 MiB | 33.46 MiB/s, done.
Resolving deltas: 100% (402/402), done.
/content/vqa-maskrcnn-benchmark
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
running build
running build_py
creating build
creating build/lib.linux-x86_64-3.6
creating build/lib.linux-x86_64-3.6/maskrcnn_benchmark
copying maskrcnn_benchmark/__init__.py -> build/lib.linux-x86_64-3.6/maskrcnn_benchmark
creating build/lib.linux-x86_64-3.6/maskrcnn_benchmark/structures
copying maskrcnn_benchmark/structures/bounding_box.py -> build/lib.linux-x86_64-3.6/maskrcnn_benchmark/structures
copying maskrcnn_benchmark/structures/__init__.py -> build/lib.linux-x86_64-3.6/maskrcnn_benchmark/structures
copying maskrcnn_benchmark/structures/image_l

In [0]:
import yaml
import cv2
import torch
import requests
import numpy as np
import gc
import torch.nn.functional as F
import pandas as pd

import torchvision.models as models
import torchvision.transforms as transforms


from PIL import Image
from IPython.display import display, HTML, clear_output
from ipywidgets import widgets, Layout
from io import BytesIO

from maskrcnn_benchmark.config import cfg
from maskrcnn_benchmark.layers import nms
from maskrcnn_benchmark.modeling.detector import build_detection_model
from maskrcnn_benchmark.structures.image_list import to_image_list
from maskrcnn_benchmark.utils.model_serialization import load_state_dict

from pythia.utils.configuration import ConfigNode
from pythia.tasks.processors import VocabProcessor, VQAAnswerProcessor
from pythia.tasks.processors import FastTextProcessor, SoftCopyAnswerProcessor, SimpleWordProcessor
from pythia.models import LoRRA
from pythia.common.registry import registry
from pythia.common.sample import Sample, SampleList
from pprint import pprint

In [0]:
%cd /content/pythia

!ls configs/vqa/textvqa/lorra.yml

/content/pythia
configs/vqa/textvqa/lorra.yml


In [0]:
pythia_path = '/content/pythia'
lorra_model_config_path = '/content/pythia/configs/vqa/textvqa/lorra.yml'
detectron_model_config_path = '/content/configs/detectron_model/detectron_model.yaml'

In [0]:
%cd /content/

%mkdir model_data

!wget -O /content/model_data/vocabulary_100k.txt https://dl.fbaipublicfiles.com/pythia/data/vocabulary_100k.txt

/content
--2019-06-08 20:10:17--  https://dl.fbaipublicfiles.com/pythia/data/vocabulary_100k.txt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.20.6.166, 104.20.22.166, 2606:4700:10::6814:16a6, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.20.6.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 626738 (612K) [text/plain]
Saving to: ‘/content/model_data/vocabulary_100k.txt’

/content/model_data 100%[===================>] 612.05K  1.72MB/s    in 0.3s    

2019-06-08 20:10:18 (1.72 MB/s) - ‘/content/model_data/vocabulary_100k.txt’ saved [626738/626738]



In [0]:
!wget -O /content/model_data/answers_vqa.txt https://dl.fbaipublicfiles.com/pythia/data/answers_vqa.txt
!wget -O /content/model_data/vocabulary_100k.txt https://dl.fbaipublicfiles.com/pythia/data/vocabulary_100k.txt
!wget -O /content/model_data/detectron_model.pth  https://dl.fbaipublicfiles.com/pythia/detectron_model/detectron_model.pth 
!wget -O /content/model_data/pythia.pth https://dl.fbaipublicfiles.com/pythia/pretrained_models/vqa2/pythia_train_val.pth
!wget -O /content/model_data/pythia.yaml https://dl.fbaipublicfiles.com/pythia/pretrained_models/vqa2/pythia_train_val.yml
!wget -O /content/model_data/detectron_model.yaml https://dl.fbaipublicfiles.com/pythia/detectron_model/detectron_model.yaml
!wget -O /content/model_data/detectron_weights.tar.gz https://dl.fbaipublicfiles.com/pythia/data/detectron_weights.tar.gz
!tar xf /content/model_data/detectron_weights.tar.gz

--2019-06-08 20:11:03--  https://dl.fbaipublicfiles.com/pythia/data/answers_vqa.txt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.20.6.166, 104.20.22.166, 2606:4700:10::6814:16a6, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.20.6.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24768 (24K) [text/plain]
Saving to: ‘/content/model_data/answers_vqa.txt’

/content/model_data 100%[===================>]  24.19K  --.-KB/s    in 0.06s   

2019-06-08 20:11:03 (399 KB/s) - ‘/content/model_data/answers_vqa.txt’ saved [24768/24768]

--2019-06-08 20:11:04--  https://dl.fbaipublicfiles.com/pythia/data/vocabulary_100k.txt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.20.6.166, 104.20.22.166, 2606:4700:10::6814:16a6, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.20.6.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 626738 (612K) [text/plain]
Saving to: ‘/cont

In [0]:
!wget -O /content/model_data/answers_textvqa_more_than_1.txt  https://dl.fbaipublicfiles.com/pythia/data/answers_textvqa_more_than_1.txt

--2019-06-08 20:13:46--  https://dl.fbaipublicfiles.com/pythia/data/answers_textvqa_more_than_1.txt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.20.6.166, 104.20.22.166, 2606:4700:10::6814:16a6, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.20.6.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32540 (32K) [text/plain]
Saving to: ‘/content/model_data/answers_textvqa_more_than_1.txt’

/content/model_data 100%[===================>]  31.78K  --.-KB/s    in 0.07s   

2019-06-08 20:13:46 (441 KB/s) - ‘/content/model_data/answers_textvqa_more_than_1.txt’ saved [32540/32540]



In [0]:
!wget -O /content/model_data/lorra.pth https://dl.fbaipublicfiles.com/pythia/pretrained_models/textvqa/lorra_best.pth

--2019-06-08 20:18:03--  https://dl.fbaipublicfiles.com/pythia/pretrained_models/textvqa/lorra_best.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.20.6.166, 104.20.22.166, 2606:4700:10::6814:6a6, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.20.6.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 892955500 (852M) [application/octet-stream]
Saving to: ‘/content/model_data/lorra.pth’

/content/model_data 100%[===================>] 851.59M  27.5MB/s    in 32s     

2019-06-08 20:18:36 (26.7 MB/s) - ‘/content/model_data/lorra.pth’ saved [892955500/892955500]



In [0]:
class TextVQADemo:
    TARGET_IMAGE_SIZE = [448, 448]
    CHANNEL_MEAN = [0.485, 0.456, 0.406]
    CHANNEL_STD = [0.229, 0.224, 0.225]
    
    def __init__(self):
        self._init_processors()
        self.lorra_model = self._build_lorra_model()
        self.detection_model = self._build_detection_model()
        self.resnet_model = self._build_resnet_model()
        
    def _init_processors(self):
        """
        Pythia uses processors is to keep data processing pipelines as similar as
        possible for different datasets and allow code reusability.
        """
        with open("pythia/configs/vqa/textvqa/lorra.yml") as f:
            config = yaml.load(f)
            

        #update config with includes for model specific config
        for inc in config.get("includes", []):
            config.update(yaml.load(open("pythia/pythia/"+ inc)))

        config = ConfigNode(config)
        config.datasets = 'textvqa'
        config.training_parameters.evalai_inference = True
        textvqa_config = config.task_attributes.vqa.dataset_attributes.textvqa
        
        answer_processor_config = textvqa_config.processors.answer_processor
        answer_processor_config.params.vocab_file = \
            "/content/model_data/answers_textvqa_more_than_1.txt"
        self.answer_processor = SoftCopyAnswerProcessor(answer_processor_config.params)
        print("self.answer_processor.get_vocab_size()", self.answer_processor.get_vocab_size())
    
        registry.register("textvqa_num_final_outputs", 
                      self.answer_processor.get_vocab_size())
        registry.register("textvqa_answer_processor", self.answer_processor)
        
        text_processor_config = textvqa_config.processors.text_processor
        text_processor_config.params.vocab.vocab_file = "/content/model_data/vocabulary_100k.txt"
        self.text_processor = VocabProcessor(text_processor_config.params)
        registry.register("textvqa_text_processor", self.text_processor)
        registry.register("textvqa_text_vocab_size", 
                      self.text_processor.get_vocab_size())
        
        ocr_token_processor_config = textvqa_config.processors.ocr_token_processor
        self.ocr_token_processor = SimpleWordProcessor(ocr_token_processor_config)
        registry.register("textvqa_ocr_token_processor", self.ocr_token_processor)
        #pprint(text_processor_config)
        
        context_processor_config = textvqa_config.processors.context_processor
        self.context_processor = FastTextProcessor(context_processor_config.params)
        registry.register("textvqa_context_processor", self.context_processor)
        
        self.config = config

        registry.register("config", config)
        #pprint(config.classifier.keys())
        
        
    def _add_imdb_details(self):
        imdb_files = self.config.imdb_files
        dataset_type = 'textvqa'
        if dataset_type not in imdb_files:
            raise ValueError(
                "Dataset type {} is not present in "
                "imdb_files of dataset config".format(dataset_type)
            )

        self.imdb_file = imdb_files[dataset_type][imdb_file_index]
        self.imdb_file = self._get_absolute_path(self.imdb_file)
        self.imdb = ImageDatabase(self.imdb_file)
        
    def _multi_gpu_state_to_single(self, state_dict):
        new_sd = {}
        for k, v in state_dict.items():
            if not k.startswith('module.'):
                raise TypeError("Not a multiple GPU state of dict")
            k1 = k[7:]
            new_sd[k1] = v
        return new_sd
    

    def _build_lorra_model(self):
        state_dict = torch.load('/content/model_data/lorra.pth')
        #pprint(state_dict)
        model_config = self.config.model_attributes.lorra
        pprint(self.config.datasets)
        model = LoRRA(model_config)
        model.build()
        model.init_losses_and_metrics()
        self.model = model
        #print(model.params)
        if list(state_dict.keys())[0].startswith('module') and \
           not hasattr(model, 'module'):
            state_dict = self._multi_gpu_state_to_single(state_dict)
        model.load_state_dict(state_dict)
        model.to("cuda")
        model.eval()
    
        return model
    
    def _build_resnet_model(self):
        self.data_transforms = transforms.Compose([
            transforms.Resize(self.TARGET_IMAGE_SIZE),
            transforms.ToTensor(),
            transforms.Normalize(self.CHANNEL_MEAN, self.CHANNEL_STD),
        ])
        resnet152 = models.resnet152(pretrained=True)
        resnet152.eval()
        modules = list(resnet152.children())[:-2]
        self.resnet152_model = torch.nn.Sequential(*modules)
        self.resnet152_model.to("cuda")
        
    def _build_detection_model(self):
        cfg.merge_from_file('/content/pythia/configs/detectron_model/detectron_model.yaml')
        cfg.freeze()
        model = build_detection_model(cfg)
        checkpoint = torch.load('data/detectron/model/detectron_model.pth', 
                                  map_location=torch.device("cpu"))
        load_state_dict(model, checkpoint.pop("model"))

        model.to("cuda")
        model.eval()
        return model
    
    
    def add_ocr_details(self, sample_info, sample):
        if self.use_ocr:
            # Preprocess OCR tokens
            ocr_tokens = [
                self.ocr_token_processor({"text": token})["text"]
                for token in sample_info["ocr_tokens"]
            ]
            # Get embeddings for tokens
            context = self.context_processor({"tokens": ocr_tokens})
            sample.context = context["text"]
            sample.context_tokens = context["tokens"]
            sample.context_feature_0 = context["text"]
            sample.context_info_0 = Sample()
            sample.context_info_0.max_features = context["length"]

            order_vectors = torch.eye(len(sample.context_tokens))
            order_vectors[context["length"] :] = 0
            sample.order_vectors = order_vectors

        if self.use_ocr_info and "ocr_info" in sample_info:
            sample.ocr_bbox = self.bbox_processor({"info": sample_info["ocr_info"]})[
                "bbox"
            ]

        return sample
    
    def predict(self, url, question):
        with torch.no_grad():
            detectron_features = self.get_detectron_features(url)
            resnet_features = self.get_resnet_features(url)
        
            sample = Sample()

            processed_text = self.text_processor({"text": question})
            sample.text = processed_text["text"]
            sample.text_len = len(processed_text["tokens"])

            sample.image_feature_0 = detectron_features
            sample.image_info_0 = Sample({
              "max_features": torch.tensor(100, dtype=torch.long)
            })
            
            
            # Get embeddings for tokens
            context = self.context_processor({"tokens": processed_text["tokens"]})
            sample.context = context["text"]
            sample.context_tokens = context["tokens"]
            sample.context_feature_0 = context["text"]
            sample.context_info_0 = Sample()
            sample.context_info_0.max_features = context["length"]

            order_vectors = torch.eye(len(sample.context_tokens))
            order_vectors[context["length"] :] = 0
            sample.order_vectors = order_vectors

            sample.image_feature_1 = resnet_features

            sample_list = SampleList([sample])
            sample_list = sample_list.to("cuda")

            scores = self.lorra_model(sample_list)["scores"]
            scores = torch.nn.functional.softmax(scores, dim=1)
            actual, indices = scores.topk(5, dim=1)

            top_indices = indices[0]
            top_scores = actual[0]

            probs = []
            answers = []

            for idx, score in enumerate(top_scores):
                probs.append(score.item())
                answers.append(
                    self.answer_processor.idx2word(top_indices[idx].item())
                )

        gc.collect()
        torch.cuda.empty_cache()
        return probs, answers
    
    def get_actual_image(self, image_path):
        if image_path.startswith('http'):
            path = requests.get(image_path, stream=True).raw
        else:
            path = image_path
        return path

    def _image_transform(self, image_path):
        path = self.get_actual_image(image_path)

        img = Image.open(path)
        im = np.array(img).astype(np.float32)
        im = im[:, :, ::-1]
        im -= np.array([102.9801, 115.9465, 122.7717])
        im_shape = im.shape
        im_size_min = np.min(im_shape[0:2])
        im_size_max = np.max(im_shape[0:2])
        im_scale = float(800) / float(im_size_min)
        # Prevent the biggest axis from being more than max_size
        if np.round(im_scale * im_size_max) > 1333:
            im_scale = float(1333) / float(im_size_max)
        im = cv2.resize(
           im,
           None,
           None,
           fx=im_scale,
           fy=im_scale,
           interpolation=cv2.INTER_LINEAR
           )
        img = torch.from_numpy(im).permute(2, 0, 1)
        return img, im_scale
    
    def _process_feature_extraction(self, output,
                                    im_scales,
                                    feat_name='fc6',
                                    conf_thresh=0.2):
        batch_size = len(output[0]["proposals"])
        n_boxes_per_image = [len(_) for _ in output[0]["proposals"]]
        score_list = output[0]["scores"].split(n_boxes_per_image)
        score_list = [torch.nn.functional.softmax(x, -1) for x in score_list]
        feats = output[0][feat_name].split(n_boxes_per_image)
        cur_device = score_list[0].device

        feat_list = []

        for i in range(batch_size):
            dets = output[0]["proposals"][i].bbox / im_scales[i]
            scores = score_list[i]

            max_conf = torch.zeros((scores.shape[0])).to(cur_device)

            for cls_ind in range(1, scores.shape[1]):
                cls_scores = scores[:, cls_ind]
                keep = nms(dets, cls_scores, 0.5)
                max_conf[keep] = torch.where(cls_scores[keep] > max_conf[keep],
                                            cls_scores[keep],
                                            max_conf[keep])

            keep_boxes = torch.argsort(max_conf, descending=True)[:100]
            feat_list.append(feats[i][keep_boxes])
        return feat_list

    def masked_unk_softmax(self, x, dim, mask_idx):
        x1 = F.softmax(x, dim=dim)
        x1[:, mask_idx] = 0
        x1_sum = torch.sum(x1, dim=1, keepdim=True)
        y = x1 / x1_sum
        return y

    def get_resnet_features(self, image_path):
        path = self.get_actual_image(image_path)
        img = Image.open(path).convert("RGB")
        img_transform = self.data_transforms(img)
        
        if img_transform.shape[0] == 1:
            img_transform = img_transform.expand(3, -1, -1)
        img_transform = img_transform.unsqueeze(0).to("cuda")
        
        features = self.resnet152_model(img_transform).permute(0, 2, 3, 1)
        features = features.view(196, 2048)
        return features

    def get_detectron_features(self, image_path):
        im, im_scale = self._image_transform(image_path)
        img_tensor, im_scales = [im], [im_scale]
        current_img_list = to_image_list(img_tensor, size_divisible=32)
        current_img_list = current_img_list.to('cuda')
        with torch.no_grad():
            output = self.detection_model(current_img_list)
        feat_list = self._process_feature_extraction(output, im_scales, 
                                                    'fc6', 0.2)
        return feat_list[0]
      
    def get_ocr_tokens(self, image_path):
        

In [0]:
!apt-get install tesseract-ocr
!pip install pytesseract



In [0]:
demo = TextVQADemo()

/content/pythia/pythia/tasks/processors.py:690: UserWarning: 'max_length' not defined in the config. Setting to default of 50
  "Setting to default of {}".format(self.DEFAULT_MAX_LENGTH)
/content/pythia/pythia/.vector_cache/glove.6B.zip: 0.00B [00:00, ?B/s]

self.answer_processor.get_vocab_size() 4047


/content/pythia/pythia/.vector_cache/glove.6B.zip: 862MB [00:53, 16.2MB/s]                           
100%|█████████▉| 399916/400000 [00:44<00:00, 8900.23it/s]

FileNotFoundError: ignored

In [0]:
def init_widgets(url, question):
    image_text = widgets.Text(
        description="Image URL", layout=Layout(minwidth="70%")
    )
    question_text = widgets.Text(
        description="Question", layout=Layout(minwidth="70%")
    )

    image_text.value = url
    question_text.value = question
    submit_button = widgets.Button(description="Ask TextVQA!")

    display(image_text)
    display(question_text)
    display(submit_button)

    submit_button.on_click(lambda b: on_button_click(
      b, image_text, question_text
    ))

    return image_text, question_text

In [0]:
def on_button_click(b, image_text, question_text):
    clear_output()
    image_path = demo.get_actual_image(image_text.value)
    image = Image.open(image_path)
  
    scores, predictions = demo.predict(image_text.value, question_text.value)
    scores = [score * 100 for score in scores]
    df = pd.DataFrame({
      "Prediction": predictions,
      "Confidence": scores
    })
  
    init_widgets(image_text.value, question_text.value)
    display(image)

    display(HTML(df.to_html()))

In [0]:
image_text, question_text = init_widgets(
    "http://images.cocodataset.org/train2017/000000505539.jpg", 
    "where is this place?"
)


IndexError: list index out of range